In [ ]:
. ../../../../scripts/nbs_header.ps1
. ../../../../scripts/core.ps1

In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --execute-command "pwsh -c `"../../../../scripts/invoke-dib.ps1 test.dib`"" } | Invoke-Block -Retries 5

00:00:00 #1 [debug] runWithTimeoutChildAsync / timeout: 60
00:00:00 #2 [debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:main@441-43> }
00:00:00 #3 [verbose] > pwd: C:\home\git\polyglot\apps\spiral\temp\test
00:00:00 #4 [verbose] > dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:00 #5 [verbose] > targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:00:00 #6 [verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:00 #7 [debug] runWithTimeoutChildAsync / timeout: 500
00:00:00 #8 [verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:01 #9 [debug] sendJson / port: 13805 / json: {"Pin

In [ ]:
{ . ../../../../apps/parser/dist/DibParser$(GetExecutableSuffix) test.dib spi } | Invoke-Block

00:00:00 #1 [debug] writeDibCode / output: Spi / path: test.dib
00:00:00 #2 [debug] parseDibCode / output: Spi / file: test.dib


In [ ]:
{ . ../../../../apps/spiral/dist/Supervisor$(GetExecutableSuffix) --build-file test.spi test.fsx --timeout 10000 } | Invoke-Block

00:00:00 #1 [debug] runWithTimeoutChildAsync / timeout: 60
00:00:00 #2 [debug] executeAsync / options: { Command =
   "dotnet "C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release\Spiral.dll" --port 13805 --default-int i32 --default-float f64"
  WorkingDirectory = None
  CancellationToken = Some System.Threading.CancellationToken
  OnLine = Some <fun:main@441-43> }
00:00:00 #3 [verbose] > pwd: C:\home\git\polyglot\apps\spiral\temp\test
00:00:00 #4 [verbose] > dllPath: C:\home\git\polyglot\deps\The-Spiral-Language\The Spiral Language 2\artifacts\bin\The Spiral Language 2\release
00:00:00 #5 [verbose] > targetDir: C:\home\git\polyglot\target/polyglot/spiral_eval
00:00:00 #6 [verbose] > Starting the Spiral Server. It is bound to: http://localhost:13805
00:00:00 #7 [debug] runWithTimeoutChildAsync / timeout: 500
00:00:00 #8 [verbose] waitForPortAccess / port: 13805 / retry: 0
00:00:01 #9 [debug] sendJson / port: 13805 / json: {"Pin

In [ ]:
dotnet fable --optimize --lang rs --extension .rs

Fable 4.14.0: F# to Rust compiler (status: alpha)

Thanks to the contributor! @byte-666
Stand with Ukraine! https://standwithukraine.com.ua/

Parsing test.fsproj...
Retrieving project options from cache, in case of issues run `dotnet fable clean` or try `--noCache` option.
Project and references (1 source files) parsed in 196ms

Started Fable compilation...

Fable compilation finished in 1409ms

.\test.fsx(6,0): (6,2) warning FABLE: For Rust, support for F# static and module do bindings is disabled by default. It can be enabled with the 'static_do_bindings' feature. Use at your own risk!


In [ ]:
(Get-Content test.rs) `
    -replace [regex]::Escape("),);"), "));" `
| Set-Content test.rs

In [ ]:
cargo fmt --

In [ ]:
cargo build --release

   Compiling getrandom v0.2.14
   Compiling rand_core v0.6.4
   Compiling syn v2.0.58
   Compiling rand_chacha v0.3.1
   Compiling rand_xorshift v0.3.0
   Compiling rand v0.8.5
   Compiling proptest v1.4.0
   Compiling thiserror-impl v1.0.58
   Compiling tracing-attributes v0.1.27
   Compiling thiserror v1.0.58
   Compiling tracing v0.1.40
   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
  --> apps\spiral\temp\test\./main.rs:11:27
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                           ^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `any` is already imported here
   |
   = note: `#[warn(unused_imports)]` on by default

  --> apps\spiral\temp\test\./main.rs:11:32
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                                ^^^^^^^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `Arbitrary` is already imported here

   --> apps\spiral\temp\test\./main

In [ ]:
$env:RUST_LOG="info"
{ cargo test --release } | Invoke-Block

   Compiling rand v0.8.5
   Compiling rand_xorshift v0.3.0
   Compiling thiserror v1.0.58
   Compiling tracing v0.1.40
   Compiling proptest v1.4.0
   Compiling spiral_temp_test v0.0.1 (C:\home\git\polyglot\apps\spiral\temp\test)
  --> apps\spiral\temp\test\./main.rs:11:27
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                           ^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `any` is already imported here
   |
   = note: `#[warn(unused_imports)]` on by default

  --> apps\spiral\temp\test\./main.rs:11:32
   |
11 | use proptest::arbitrary::{any, Arbitrary};
   |                                ^^^^^^^^^
12 | use proptest::prelude::*;
   |     -------------------- the item `Arbitrary` is already imported here

    Finished `release` profile [optimized] target(s) in 30.45s
     Running unittests main.rs (C:\home\git\polyglot\target\release\deps\spiral_temp_test-f37c8ee95c038f68.exe)

running 3 tests
test test_parse_number ... ok
2024-04

In [ ]:
{ . $ScriptDir/../../../../target/release/spiral_temp_test$(GetExecutableSuffix) } | Invoke-Block

app=test
